## (WORKSHOP) LOGDATA분석

In [121]:
# accesslog.json 파일을 읽어서 t값을 timestamp값을 구한 후
# 1) t의 값을 yyyymmdd형태로 출력해보세요. min, max를 구하면 어떤 결과가 나오나요?
# 2) tz의 중복을 제거한 목록을 만들어보세요. Asia에 몇개 도시가 있나요?
# 3) a(gent) key값의 OS별(Windows, Macintosh, Linux) 접속 count를 구해보세요.

from datetime import datetime
from collections import Counter
import json

s = [
    datetime.fromtimestamp(json.loads(line).get('t')).strftime('%Y%m%d')
    for line in open('data/accesslog.json', encoding='utf8')
    if json.loads(line).get('t')
]
min(s), max(s)

s = {
    json.loads(line).get('tz')
    for line in open('data/accesslog.json', encoding='utf8')
    if json.loads(line).get('tz')
    if json.loads(line).get('tz').startswith('Asia')
}
s

s = Counter([
    json.loads(line).get('a')
    for line in open('data/accesslog.json', encoding='utf8')
    if json.loads(line).get('a')
])
s.most_common(2)

[('Mozilla/5.0 (Windows NT 6.1; WOW64; rv:10.0.2) Gecko/20100101 Firefox/10.0.2',
  309),
 ('Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.79 Safari/535.11',
  229)]

## (WORKSHOP) BIGFILE 재처리

In [122]:
# bigfile.json 파일을 읽어서 영양분(nutrients) 정보만
# 재처리 후 description 순서로 출력하는 함수를 만듭니다.
# sorted, sort를 이용해보고 빠른 성능을 보이는 방법을 택하세요!

# < result >
# description,group,units,value
# Protein,Composition,g,25.18
# ...
data = json.load(open('data/bigfile.json'))

In [130]:
# class설계하거나 namedtuple을 활용하세요
from collections import namedtuple
Nutrient = namedtuple('Nutrient', 'description,group,units,value')
# n = Nutrient(**data[0]['nutrients'][0])

# class Nutrient:
#     def __init__(self, **kwargs):
#         pass
#     @classmethod
#     def from_dict(cls, d):
#         return cls(**d)
    
[Nutrient(**d) for ds in data for d in ds['nutrients']][:3]
# len({Nutrient.from_dict(d) for ds in data for d in ds['nutrients']}), \
# len([Nutrient.from_dict(d) for ds in data for d in ds['nutrients']])
# s = {Nutrient.from_dict(d) for ds in data for d in ds['nutrients']}
# s = sorted([Nutrient.from_dict(d) for ds in data for d in ds['nutrients']], key=lambda x:x.description)
# s

[Nutrient(description='Protein', group='Composition', units='g', value=25.18),
 Nutrient(description='Total lipid (fat)', group='Composition', units='g', value=29.2),
 Nutrient(description='Carbohydrate, by difference', group='Composition', units='g', value=3.06)]

In [125]:
s = set([Nutrient.from_dict(d).units for ds in data for d in ds['nutrients']])
s

AttributeError: type object 'Nutrient' has no attribute 'from_dict'

## (WORKSHOP) 데이터 선택적 처리

연도별 사람들의 출생신고 빈도수가 적혀있는 데이터셋에서 '특정이름'의 빈도수 합계를 계산하는 함수를 만들어주세요.

데이터특징
- 각 파일의 이름/성별은 고유합니다.
- 파일이름에 연도가 있습니다.

함수요구사항
- return값은 (빈도수,엔트리검색개수)를 tuple로 반납합니다.
- 함수의 파라미터 설계를 해주세요.
 - 검색을 원하는 사용자의 fullname을 mandatory로 받습니다. (대소문자 구분없음)
 - 남자/여자 구분을 mandatory로 받도록(M/F)
 - 시작연도와 종료연도를 option으로 받도록(1880-2010)
 - keyword only로 log출력여부를 받도록(True/False)
- 내부적으로 threading사용가능합니다. (optional)

In [ ]:

def search(search_name, gender='M',
           year_range=(1880, 2010), *, log=False):
    import pdb; pdb.set_trace()
    def _proc(year):
        for index, line in enumerate(
                open('data/names/yob{}.txt'.format(year))):
            n, g, freq = line.split(',')
            if n == search_name and g == gender:
                return (int(freq), index+1)
        return 0, index + 1
    result = list(map(
        _proc,
        range(year_range[0], year_range[1])
    ))
    return (
        sum(x[0] for x in result),
        sum(x[1] for x in result),
    )

search('John', 'M')

> <ipython-input-137-6fbb25a304ef>(5)search()
-> def _proc(year):
(Pdb) search_name
'John'
(Pdb) year_range
(1880, 2010)
(Pdb) 3+2
5
(Pdb) help

Documented commands (type help <topic>):
EOF    c          d        h         list      q        rv       undisplay
a      cl         debug    help      ll        quit     s        unt      
alias  clear      disable  ignore    longlist  r        source   until    
args   commands   display  interact  n         restart  step     up       
b      condition  down     j         next      return   tbreak   w        
break  cont       enable   jump      p         retval   u        whatis   
bt     continue   exit     l         pp        run      unalias  where    

Miscellaneous help topics:
exec  pdb



## (WORKSHOP) 문자열 조합 경우의 수

단어 조합을 만드는 기능을 구현하고자 합니다. 각각을 클래스/함수 각각의 버전으로 만들어보세요
- 한글 이름 테스트 -> ['한글이름테스트', '한글이름 테스트', '한글 이름테스트', '한글 이름 테스트']
- 한글 테스트 -> ['한글테스트', '한글 테스트']
- 한글테스트 -> ['한글테스트']

In [ ]:
# class version
# wc = WordCombination()
# wc('한글', '이름', '테스트')

# function version
# wc = word_combination()
# wc('한글', '이름', '테스트')

# core-logic
from itertools import product, zip_longest
from functools import reduce
from operator import add

words = 'a b c'.split()
[
    ''.join(reduce(add, zip_longest(words, concat_str, fillvalue='')))
    for concat_str in product(['', ' '], repeat=len(words)-1)
]
